# Import Modules

In [1]:
from Dataload import dataload

import pandas as pd

from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils
from constant import EMOTIPATH
from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np
from torchvision import transforms
import os

new_base_path="../../../hdd/Dataset/Dataset/Face_Cropped2/"


<!-- os.mkdir(new_base_path)

os.mkdir(os.path.join(new_base_path,"Train"))
os.mkdir(os.path.join(new_base_path,"pt"))
os.mkdir(os.path.join(new_base_path,"pt","Train"))
os.mkdir(os.path.join(new_base_path,"Valid"))
os.mkdir(os.path.join(new_base_path,"pt","Valid")   -->

In [13]:
class Raw_Data():
    def __init__(self,csv_file,sub_csv_file=None,
                 base_path_v=None,frame_num=25,embedding=False):
        
        self.max_frame_num=25
        self._table = pd.read_csv(csv_file,delimiter=' ')
        if sub_csv_file is None:
            self._table_embedding=None
        else:
            self._table_embedding=pd.read_csv(sub_csv_file)
        self.frame_num = frame_num
        self._base_path_v=base_path_v
        self.embedding=embedding
        self.transform=transforms.Compose([
                     transforms.Resize((256,256)),
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
        self.endPad=self.transform(Image.new(mode='RGB', size=(256,256), color=0))    

       
    def __len__(self):
        return len(self._table)

    def __getitem__(self, idx):
        folder_name = os.path.join(self._base_path_v,self._table.Vid_name[idx])
        first=True
            
        if self._table_embedding is not None:
            temp_frame_embedding=torch.from_numpy(np.array(self._table_embedding.Embedding[idx].split(),dtype=float)).reshape((self.max_frame_num,-1))
            frame_data=torch.empty(size=(self.frame_num,1000),dtype=torch.double)
            if self.frame_num<25:
                index=np.linspace(0,23,self.frame_num,dtype=int)
 
                for i,copy in enumerate(index):
                    frame_data[i]=temp_frame_embedding[copy]
            return (frame_data,audio_img,labels)
    

            
        frame_raw_list=os.listdir(folder_name)
        frame_len=len(frame_raw_list)

        frame_raw_list=sorted(frame_raw_list)
       # print(frame_raw_list)
        frame_list=[]
        
        if frame_len<self.frame_num:
            for index_0 in range(frame_len):
                frame_path=os.path.join(folder_name,frame_raw_list[index_0])               
                tempimg=Image.open(frame_path)       
                frame_list.append(self.transform(tempimg))
            
        else:    
            frame_index=(np.linspace(0,frame_len-1,self.frame_num,dtype=int))

            for index_2 in frame_index:
                frame_path=os.path.join(folder_name,frame_raw_list[index_2])
                tempimg=Image.open(frame_path)
                frame_list.append(self.transform(tempimg))
        while(len(frame_list)<self.frame_num):
            frame_list.append(self.endPad)
        frame_data=torch.stack(frame_list,dim=0)
            
        return (folder_name,frame_data,frame_raw_list)
    

def face_extraction(img,max_number):
        def toonePIL(x,max_number):
            new_im = Image.new('RGB', (64*max_number,64))

            x_offset = 0
            for im in x:
                new_im.paste(im, (x_offset,0))
                x_offset += im.size[0]
            return new_im

        bounding_boxes, landmarks = detect_faces(img)
        img_list=[]
        transform=transforms.Resize((64,64))
        transform2=transforms.Compose([
                     transforms.ToTensor(),   
                     transforms.Normalize((0.5,0.5,0.5 ), (0.5, 0.5,0.5))])
        for box_index,(left,right,up,bottom,_) in enumerate(bounding_boxes):
            cropped_img=img.crop((left,right,up,bottom))
            img_list.append(transform(cropped_img))
            
            if len(img_list)==max_number:
                break
        while len(img_list) !=max_number:
            END_PAD= Image.new(mode = "RGB", size = (64, 64), color =(0, 0, 0))
            img_list.append(END_PAD)
        img=toonePIL(img_list,max_number)
        pt=transform2(img)
        return img,pt
    
    
    
    
    
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")
train_data=Raw_Data(Train_label,base_path_v=Train_video,frame_num=25)
valid_data=Raw_Data(Val_labels,base_path_v=Val_video,frame_num=25)


save_image_path=os.path.join(new_base_path,"Train",os.path.basename(video_name))
save_pt_path=os.path.join(new_base_path,"pt","Train",os.path.basename(video_name))

In [ ]:
##os.mkdir('../../../hdd/Dataset/Dataset/Face_Cropped/Val')

def create_face(data,save_pt_path,save_image_path:)
    for index in tqdm(data):
        video_name,_,frame_list=index
        for frame_name in frame_list:

            path=os.path.join(video_name,frame_name)
            img=Image.open(path)
            try:
                face_cropped,pt=face_extraction(img,5)
            except:
                face_cropped=Image.new(mode='RGB', size=(320,64), color=0)
                pt=transform2(face_cropped)
                print(video_name,frame_name)
                continue
            
            if not os.path.isdir(save_image_path):
                os.mkdir(save_image_path)
            if not os.path.isdir(save_pt_path):
                os.mkdir(save_pt_path)
            face_cropped.save((os.path.join(save_image_path,frame_name)))
            torch.save(pt,(os.path.join(save_pt_path,os.path.splitext(frame_name)[0]+'.pt')))
        
